In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', 200)

In [7]:
data = pd.read_csv('data/clean_data.csv')
data.head()

,hero_names,Gender,Eye color,Race,Hair color,Height,Publisher,Alignment,Weight,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,Danger Sense,Underwater breathing,Marksmanship,Weapons Master,Power Augmentation,Animal Attributes,Longevity,Intelligence,Super Strength,Cryokinesis,Telepathy,Energy Armor,Energy Blasts,Duplication,Size Changing,Density Control,Stamina,Astral Travel,Audio Control,Dexterity,Omnitrix,Super Speed,Possession,Animal Oriented Powers,Weapon-based Powers,Electrokinesis,Darkforce Manipulation,Death Touch,Teleportation,Enhanced Senses,Telekinesis,Energy Beams,Magic,Hyperkinesis,Jump,Clairvoyance,Dimensional Travel,Power Sense,Shapeshifting,Peak Human Condition,Immortality,Camouflage,Element Control,Phasing,Astral Projection,Electrical Transport,Fire Control,Projection,Summoning,Enhanced Memory,Reflexes,Invulnerability,Energy Constructs,Force Fields,Self-Sustenance,Anti-Gravity,Empathy,Power Nullifier,Radiation Control,Psionic Powers,Elasticity,Substance Secretion,Elemental Transmogrification,Technopath/Cyberpath,Photographic Reflexes,Seismic Power,Animation,Precognition,Mind Control,Fire Resistance,Power Absorption,Enhanced Hearing,Nova Force,Insanity,Hypnokinesis,Animal Control,Natural Armor,Intangibility,Enhanced Sight,Molecular Manipulation,Heat Generation,Adaptation,Gliding,Power Suit,Mind Blast,Probability Manipulation,Gravity Control,Regeneration,Light Control,Echolocation,Levitation,Toxin and Disease Control,Banish,Energy Manipulation,Heat Resistance,Natural Weapons,Time Travel,Enhanced Smell,Illusions,Thirstokinesis,Hair Manipulation,Illumination,Omnipotent,Cloaking,Changing Armor,Power Cosmic,Biokinesis,Water Control,Radiation Immunity,Vision - Telescopic,Toxin and Disease Resistance,Spatial Awareness,Energy Resistance,Telepathy Resistance,Molecular Combustion,Omnilingualism,Portal Creation,Magnetism,Mind Control Resistance,Plant Control,Sonar,Sonic Scream,Time Manipulation,Enhanced Touch,Magic Resistance,Invisibility,Sub-Mariner,Radiation Absorption,Intuitive aptitude,Vision - Microscopic,Melting,Wind Control,Super Breath,Wallcrawling,Vision - Night,Vision - Infrared,Grim Reaping,Matter Absorption,The Force,Resurrection,Terrakinesis,Vision - Heat,Vitakinesis,Radar Sense,Qwardian Power Ring,Weather Control,Vision - X-Ray,Vision - Thermal,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,good,441.0,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,good,65.0,True,True,False,False,True,True,False,False,False,False,True,True,True,False,False,True,True,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,

In [8]:
data.shape

(684, 176)

In [9]:
data.drop("hero_names", axis=1, inplace=True)

In [10]:
data = data.replace(['Male', 'Female'], [0,1])

In [11]:
data = pd.get_dummies(data)
data.shape

(684, 308)

In [12]:
types = set(data.dtypes)
types

{dtype('bool'), dtype('uint8'), dtype('int64'), dtype('float64')}

In [13]:
def split_data(data, label):
    y = data[label]
    X = data.drop([label], axis=1)
    # 75% training and 25% test
    return train_test_split(X, y, test_size=0.25, random_state=1)

In [14]:
# 75% training and 25% test
X_train, X_test, y_train, y_test = split_data(data,'Flight') 
print(X_train.shape)
X_test.shape

(513, 307)


(171, 307)

In [15]:
clf = DecisionTreeClassifier(random_state=1)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Test F1 Score:", f1_score(y_test, y_pred))
y_pred = clf.predict(X_train)
print("Train F1 Score:", f1_score(y_train, y_pred))


Test F1 Score: 0.5263157894736842
Train F1 Score: 1.0


In [17]:
def find_best_parameters(X_train, X_test, y_train, y_test):
    scorer = make_scorer(f1_score, average='binary')
    clf = DecisionTreeClassifier(random_state=1)

    parameters = {'criterion':('gini', 'entropy'), 
                  'min_samples_split':[2, 3, 4, 6, 8, 10], 
                  'max_depth':[10, 12, 14, 16, 18, 20, 22, None],
                  'min_samples_leaf':[1,2,3]
                 }

    grid = GridSearchCV(clf, parameters, scoring=scorer, cv=3)
    grid = grid.fit(X_train, y_train)
    return grid

In [18]:
grid = find_best_parameters(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [19]:
best_parameters = grid.best_params_
best_parameters

{'criterion': 'gini',
 'max_depth': 12,
 'min_samples_leaf': 1,
 'min_samples_split': 8}

In [20]:
grid.cv_results_['mean_test_score']

array([0.5985125 , 0.58812215, 0.58304542, 0.59367275, 0.57231235,
       0.56067791, 0.56227021, 0.56227021, 0.56227021, 0.55447261,
       0.55385477, 0.54747336, 0.50098561, 0.50098561, 0.50098561,
       0.50098561, 0.50455936, 0.51340208, 0.61140332, 0.6000215 ,
       0.61839752, 0.6200061 , 0.6358605 , 0.60870525, 0.58455913,
       0.58455913, 0.58455913, 0.53519061, 0.5487867 , 0.54541365,
       0.53689625, 0.53689625, 0.53689625, 0.53689625, 0.52509492,
       0.51399261, 0.62223094, 0.62296637, 0.58964077, 0.61833052,
       0.59240887, 0.57317629, 0.58109144, 0.58109144, 0.58109144,
       0.55302136, 0.54847018, 0.53479491, 0.50190355, 0.50190355,
       0.50190355, 0.50190355, 0.54305847, 0.52430794, 0.62342474,
       0.60355115, 0.59900672, 0.5972073 , 0.63115796, 0.58961752,
       0.51812095, 0.51812095, 0.51812095, 0.5543214 , 0.53872716,
       0.57212476, 0.52502088, 0.52502088, 0.52502088, 0.52502088,
       0.52269738, 0.51726387, 0.61859452, 0.59798129, 0.58102

In [21]:
grid.cv_results_['std_test_score']

array([0.0200887 , 0.03342531, 0.02142336, 0.02972617, 0.03856389,
       0.04879905, 0.0286969 , 0.0286969 , 0.0286969 , 0.04027836,
       0.05545163, 0.05965406, 0.04896832, 0.04896832, 0.04896832,
       0.04896832, 0.04029627, 0.04436616, 0.01204716, 0.02753931,
       0.00953288, 0.03239093, 0.02686255, 0.00619081, 0.01401296,
       0.01401296, 0.01401296, 0.05949282, 0.0336751 , 0.04257626,
       0.0555217 , 0.0555217 , 0.0555217 , 0.0555217 , 0.05374527,
       0.04099788, 0.01927746, 0.01826842, 0.02153019, 0.00654793,
       0.03004513, 0.03489556, 0.02121295, 0.02121295, 0.02121295,
       0.02569714, 0.04404503, 0.07482793, 0.06100625, 0.06100625,
       0.06100625, 0.06100625, 0.04673842, 0.04332521, 0.02462199,
       0.02313851, 0.02462121, 0.02360445, 0.01315424, 0.03112116,
       0.04030711, 0.04030711, 0.04030711, 0.04066746, 0.05457533,
       0.05168986, 0.07081094, 0.07081094, 0.07081094, 0.07081094,
       0.05546251, 0.04015098, 0.02325376, 0.01664861, 0.02934

In [22]:
grid.best_score_

0.6358604983305348

In [23]:
clf = grid.best_estimator_
y_pred = clf.predict(X_test)
print("F1 Score:", f1_score(y_test, y_pred))
y_pred = clf.predict(X_train)
print("Train F1 Score:", f1_score(y_train, y_pred))

F1 Score: 0.5333333333333332
Train F1 Score: 0.8231046931407944


In [26]:
clf = DecisionTreeClassifier(criterion = 'gini', max_depth = 16, min_samples_leaf = 1,
                             min_samples_split = 8, random_state = 1)
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_train)
print("Train F1 Score:", f1_score(y_train, y_pred))
y_pred = clf.predict(X_test)
print("F1 Score:", f1_score(y_test, y_pred))

Train F1 Score: 0.842857142857143
F1 Score: 0.5384615384615384


In [27]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(95, 9, 39, 28)

## Usando mais correlacionados

In [28]:
corr = data.corr()

In [29]:
flight_corr = corr.abs()
flight_corr = flight_corr.sort_values('Flight', ascending=False).head(60)['Flight']
most_corr = list(flight_corr.index)

In [30]:
most_corr

['Flight',
 'Energy Blasts',
 'Force Fields',
 'Vision - X-Ray',
 'Energy Manipulation',
 'Energy Absorption',
 'Super Speed',
 'Invulnerability',
 'Energy Constructs',
 'Phasing',
 'Molecular Manipulation',
 'Super Strength',
 'Super Breath',
 'Vision - Heat',
 'Vision - Telescopic',
 'Vision - Microscopic',
 'Invisibility',
 'Energy Beams',
 'Teleportation',
 'Time Travel',
 'Intangibility',
 'Fire Control',
 'Weapons Master',
 'Telekinesis',
 'Enhanced Hearing',
 'Reality Warping',
 'Immortality',
 'Self-Sustenance',
 'Publisher_DC Comics',
 'Telepathy',
 'Gravity Control',
 'Levitation',
 'Enhanced Senses',
 'Race_Kryptonian',
 'Stealth',
 'Time Manipulation',
 'Accelerated Healing',
 'Magnetism',
 'Lantern Power Ring',
 'Astral Projection',
 'Heat Generation',
 'Peak Human Condition',
 'Dimensional Awareness',
 'Longevity',
 'Gliding',
 'Race_God / Eternal',
 'Power Cosmic',
 'Vision - Thermal',
 'Race_Human',
 'Dimensional Travel',
 'Element Control',
 'Race_Cosmic Entity',
 'Ast

In [31]:
X_train, X_test, y_train, y_test = split_data(data[most_corr],'Flight') 

In [32]:
grid = find_best_parameters(X_train, X_test, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [33]:
grid.best_score_

0.586457953019769

In [34]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 12,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [35]:
clf = DecisionTreeClassifier(criterion = 'entropy',
 max_depth = 15,
 min_samples_leaf = 1,
 min_samples_split = 6,
 random_state = 1)

clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_train)
print("Train F1 Score:", f1_score(y_train, y_pred))
y_pred = clf.predict(X_test)
print("F1 Score:", f1_score(y_test, y_pred))

Train F1 Score: 0.8671328671328671
F1 Score: 0.6140350877192982


In [36]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(92, 12, 32, 35)